In [307]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [308]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.royallepage.ca/en/search/homes/?search_str=Ontario%2C+ON%2C+CAN&csrfmiddlewaretoken=YrzRsRQ7hui5kwIngTSfFcjdmkY4uzkOpTtA8xAWppg2PQrYHUihHDD2uSNjEGZN&property_type=7%2C8%2C5&house_type=102%2C103%2C104&features=&listing_type=&lat=50.926163435000035&lng=-84.74492999999995&upper_lat=&upper_lng=&lower_lat=&lower_lng=&bypass=&radius=&zoom=&display_type=gallery-view&travel_time=&travel_time_min=30&travel_time_mode=drive&travel_time_congestion=&da_id=&segment_id=&tier2=False&tier2_proximity=0&address=Ontario&method=homes&address_type=province&city_name=&prov_code=ON&school_id=&min_price=0&max_price=3000000&min_leaseprice=0&max_leaseprice=5000%2B&beds=0&baths=0&transactionType=SALE&archive_status=All&archive_timespan=6&sfproperty_type%5B7%5D=7&sfproperty_type%5B8%5D=8&sfproperty_type%5B5%5D=5&sfhouse_type%5B102%5D=102&sfhouse_type%5B103%5D=103&sfhouse_type%5B104%5D=104&keyword=&sortby="

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [309]:
# Go through all pages to collect data
house_listings = []
for x in range(1, 23):
    html = browser.html
    soup = bs(html, 'html.parser')
    links = soup.find_all(class_='address-1')
    
    # Get all address listings for buying    
    print('Gathering data from page:', x)
    for link in links:
        house_listings.append(link.a['href'])
    browser.find_by_css('.next').click()

# Number of listings
len(house_listings)


Gathering data from page: 1
Gathering data from page: 2
Gathering data from page: 3
Gathering data from page: 4
Gathering data from page: 5
Gathering data from page: 6
Gathering data from page: 7
Gathering data from page: 8
Gathering data from page: 9
Gathering data from page: 10
Gathering data from page: 11
Gathering data from page: 12
Gathering data from page: 13
Gathering data from page: 14
Gathering data from page: 15
Gathering data from page: 16
Gathering data from page: 17
Gathering data from page: 18
Gathering data from page: 19
Gathering data from page: 20
Gathering data from page: 21
Gathering data from page: 22


1012

In [310]:
browser.quit()

In [463]:
# Dependencies
import requests
import json

# Import the API key
from config import apiKey

# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)


# Visit each listing and grab relevant data
house_listing_info = []
for i in range(len(house_listings)):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 
    
    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    
    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text


    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    labels = building_feature_section.find_all('span', class_='label')
    values = building_feature_section.find_all('span', class_="value")
    

    style_type = values[0].text.strip(":")
    # cooling_type = values[i].text.strip(":")
    # heating_fuel = values[i].text.strip(":")
    # heating_type = values[i].text.strip(":")
    # fire_place = values[-4].text.strip(":")
    
    # Store in dictionary
    houses['address'] = address
    houses['price'] = price
    houses['number_of_beds'] = bed_count
    houses['number_of_baths'] = bath_count
    houses['style'] = style_type
    # houses['cooling_type'] = cooling_type
    # houses['heating_fuel'] = heating_fuel
    # houses['heating_type'] = heating_type
    # houses['fire_place'] = fire_place
    
    # Build target url with address and apikey
    string_search = address.split(",")[1:]
    new_string = (",").join(string_search).strip()                           
    
    target_url = f"https://api.geoapify.com/v1/geocode/search?text={new_string}&format=json&apiKey={apiKey}"
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()

    # Get coordinate information
    lat = geo_data["results"][0]["lat"]
    lon = geo_data["results"][0]["lon"]
    
    houses['lat'] = lat
    houses['lon'] = lon
        
    print(f"Collected data for: {address}")
    house_listing_info.append(houses)

browser.quit()

Collected data for: 33 SCUFFLER DR, Vaughan, Ontario, L6A 4Y7
Collected data for: 11134 5 SIDE RD, Halton Hills, Ontario, L7G 4S6
Collected data for: 14541 NIAGARA RIVER PKWY, Niagara-on-the-Lake, Ontario, L0S 1L0
Collected data for: 15415 JANE ST, King, Ontario, L7B 1A3
Collected data for: 31 WISPI SHORE RD, Kawartha Lakes, Ontario, K9V 0K4
Collected data for: 4282 VIVIAN RD, Whitchurch-Stouffville, Ontario, L4A 1Z2
Collected data for: 57 ALAMOSA DR, Toronto, Ontario, M2J 2N8
Collected data for: 121 WATERVIEW COMMON, Oakville, Ontario, L6L 0E7
Collected data for: 3696 FERRETTI CRT, Innisfil, Ontario, L9S 0N6
Collected data for: 643 WEST OVAL Drive, Burlington, Ontario, L7T 1C1
Collected data for: 40 DEAN PL, Vaughan, Ontario, L3L 0A8
Collected data for: 48 FOOTHILLS CRES, Brampton, Ontario, L6P 4G9
Collected data for: 13066 KEELE ST, King, Ontario, L7B 1H8
Collected data for: 1041 BRANDYWINE COURT, Ottawa, Ontario, K4M 1J2
Collected data for: 8254 21/22 NOTTAWASAGA SDRD, Clearview, On

KeyboardInterrupt: 

In [552]:
wanted_features = ['Fireplace', 'Style', 'Heating Fuel']

listed_features = []
listed_values = []
for i in range(len(labels)):
    listed_features.append(labels[i].text.strip(":"))
    listed_values.append(values[i].text.strip(":"))
    
for i in range(len(wanted_features)):
    if wanted_features[i] in listed_features:
        shared_index = listed_features.index(wanted_features[i])
        value = listed_values[shared_index]
value


'Natural gas'

In [611]:
# Dependencies
import requests
import json

# Import the API key
from config import apiKey

# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)


# Visit each listing and grab relevant data
house_listing_info = []
for i in range(len(house_listings)):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 
    
    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    
    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text


    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    labels = building_feature_section.find_all('span', class_='label')
    values = building_feature_section.find_all('span', class_="value")
    
    
    wanted_features = ['Fireplace', 'Style', 'Heating Fuel', 'Cooling Type', 'Heating Type', 'Building Type']
    
    listed_features = []
    listed_values = []
    for i in range(len(labels)):
        listed_features.append(labels[i].text.strip(":"))
        listed_values.append(values[i].text.strip(":"))
        
    for i in range(len(wanted_features)):
        if wanted_features[i] in listed_features:
            shared_index = listed_features.index(wanted_features[i])
            value = listed_values[shared_index]
            
                # Store in dictionary
            houses['address'] = address
            houses['price'] = price
            houses['number_of_beds'] = bed_count
            houses['number_of_baths'] = bath_count
            houses['style'] = value
            houses['building_type'] = value
            houses['cooling_type'] = value
            houses['heating_fuel'] = value
            houses['heating_type'] = value
            houses['fire_place'] = value
        else:
            value = ""
    
    # Build target url with address and apikey
    string_search = address.split(",")[1:]
    new_string = (",").join(string_search).strip()                           
    
    target_url = f"https://api.geoapify.com/v1/geocode/search?text={new_string}&format=json&apiKey={apiKey}"
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()

    # Get coordinate information
    lat = geo_data["results"][0]["lat"]
    lon = geo_data["results"][0]["lon"]
    
    houses['lat'] = lat
    houses['lon'] = lon
        
    print(f"Collected data for: {address}")
    house_listing_info.append(houses)

browser.quit()

Collected data for: 33 SCUFFLER DR, Vaughan, Ontario, L6A 4Y7
Collected data for: 11134 5 SIDE RD, Halton Hills, Ontario, L7G 4S6


In [607]:
wanted_features = ['Fireplace', 'Style', 'Heating Fuel', 'Cooling Type', 'Heating Type', 'Building Type']
    
listed_features = []
listed_values = []
for i in range(len(labels)):
    listed_features.append(labels[i].text.strip(":"))
    listed_values.append(values[i].text.strip(":"))
    
for i in range(len(wanted_features)):
    if wanted_features[i] in listed_features:
        shared_index = listed_features.index(wanted_features[i])
        value = listed_values[shared_index]
        print(value)
    else:
        style_type = ""

Yes
Detached
Central air conditioning
Forced air
House


In [609]:

print(f"Rows of data to work with: {len(house_listing_info)}")

Rows of data to work with: 3


In [610]:
import csv

try:
    with open('./Resources/housing.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['address', 'price', 'number_of_beds', 'number_of_baths', 'style', 'cooling_type','heating_fuel', 'heating_type', 'fire_place', 'lat', 'lon'])
        writer.writeheader()
        for data in house_listing_info:
            writer.writerow(data)
except IOError:
    print("I/O error")